In [ ]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [ ]:
class CNN_Text(nn.Module):
    
    def __init__(self, args):
        super(CNN_Text, self).__init__()
        self.args = args
        
        V = args.embed_num  # 词汇表的大小
        D = args.embed_dim  # 词向量的维度
        C = args.class_num  # 类别数量
        Ci = 1  # 输入通道数
        Co = args.kernel_num  # 输出通道数（卷积核的数量）
        Ks = args.kernel_sizes  # 不同卷积核的尺寸

        self.embed = nn.Embedding(V, D)  # 词嵌入层
        self.convs = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])  # 卷积层
        self.dropout = nn.Dropout(args.dropout)  # Dropout层
        self.fc1 = nn.Linear(len(Ks) * Co, C)  # 全连接层

        if self.args.static:
            self.embed.weight.requires_grad = False  # 如果使用静态词嵌入，则不更新嵌入层的权重

    def forward(self, x):
        x = self.embed(x)  # (B, S, D)  
        x = x.unsqueeze(1)  # (B, Ci, S, D)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs]  # [(B, Co, S), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(B, Co), ...]*len(Ks)
        x = torch.cat(x, 1)
        x = self.dropout(x)  # (B, len(Ks)*Co)
        logit = self.fc1(x)  # (B, C)
        return logit

重点：
1、